# 04_temp_ndvi_by_mun (Temperature anomaly, temperature slope and NDVI slope)

This notebook explores the __data produced on notebook 03__ by municipality for each selected metropolis.

## Import libraries

In [1]:
from pathlib import Path

current_path = Path().resolve()

for parent in current_path.parents:
    if parent.name == "accesibilidad-urbana":
        project_root = parent
        break

print(project_root)

/home/jovyan/accesibilidad-urbana


In [2]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

# Correlation calc
import scipy.stats as stats

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join(project_root))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
else:
    import aup

## Notebook config

In [3]:
city = 'CDMX'
res = '11'

## Input

In [4]:
# Hexs with temperature data (Have no mun data)
temperature_gdf = gpd.read_file(str(project_root)+f"/data/external/temporal_fromjupyter/2025_temp_publication/temperature/temperature_res{res}_{city}.gpkg")

# Show
print(temperature_gdf.shape)
temperature_gdf.head(2)

(326262, 19)


,hex_id,res,temperature_mean,temperature_std,temperature_median,temperature_max,temperature_min,temperature_diff,temperature_tend,temperature_2018,temperature_2019,temperature_2020,temperature_2021,temperature_2022,temperature_2023,city,temperature_anomaly,temperature_slope,geometry
0,8b4995b81984fff,11,35.268092,6.180187,35.562500,43.671875,27.281250,16.390625,0.021635,36.120536,35.362500,35.800000,32.302083,34.197917,36.947917,CDMX,1.967899,-0.081565,"POLYGON ((2796916.249 831994.546, 2796934.314 ..."
1,8b4995b8db95fff,11,35.364662,6.510117,35.053571,44.648810,26.809524,17.839286,0.005952,36.392857,35.378571,35.328571,33.511905,34.601190,36.321429,CDMX,2.064469,-0.128741,"POLYGON ((2792221.707 835854.152, 2792239.775 ..."


In [5]:
ndvi_gdf = gpd.read_file(str(project_root)+f"/data/external/temporal_fromjupyter/2025_temp_publication/ndvi/ndvi_res{res}_{city}.gpkg")

# Show
print(ndvi_gdf.shape)
ndvi_gdf.head(2)

(326262, 19)


,hex_id,res,ndvi_mean,ndvi_std,ndvi_median,ndvi_max,ndvi_min,ndvi_diff,ndvi_tend,ndvi_2018,ndvi_2019,ndvi_2020,ndvi_2021,ndvi_2022,ndvi_2023,city,ndvi_slope,ndvi_median_rango,geometry
0,8b4995bab64dfff,11,0.397782,0.121854,0.394442,0.501281,0.287628,0.213653,-0.004383,0.523185,0.499822,0.455399,0.362545,0.267237,0.278506,CDMX,-0.057543,Mínima densidad vegetal,"POLYGON ((2792526.900 834780.806, 2792544.968 ..."
1,8b4995b8db95fff,11,0.205466,0.054287,0.210134,0.266381,0.153338,0.113042,-0.001868,0.252257,0.250008,0.227518,0.203216,0.158554,0.141246,CDMX,-0.024392,Mínima densidad vegetal,"POLYGON ((2792221.707 835854.152, 2792239.775 ..."


In [6]:
# Hexs from database (have mun data)
hexs_schema = 'hexgrid'
hexs_table = f'hexgrid_{res}_city_2020'
query = f"SELECT * FROM {hexs_schema}.{hexs_table} WHERE \"city\" = '{city}\'"
hexs_gdf = aup.gdf_from_query(query, geometry_col='geometry')

# Show
print(hexs_gdf.shape)
hexs_gdf.head(2)

(604486, 6)


,hex_id_11,geometry,CVEGEO,NOMGEO,city,type
0,8b4995b813aefff,"POLYGON ((-99.16561 19.47533, -99.16543 19.475...",09002,Azcapotzalco,CDMX,urban
1,8b4995b9d922fff,"POLYGON ((-99.14093 19.49097, -99.14075 19.491...",09005,Gustavo A. Madero,CDMX,urban


## Merge

In [7]:
temp_mun_gdf = pd.merge(temperature_gdf,hexs_gdf[['hex_id_11','NOMGEO']], left_on='hex_id', right_on='hex_id_11')
temp_mun_gdf.drop(columns=['hex_id_11'],inplace=True)
temp_mun_gdf.rename(columns={'NOMGEO':'nomgeo'},inplace=True)

# Show
print(temp_mun_gdf.shape)
temp_mun_gdf.head(2)

(326262, 20)


,hex_id,res,temperature_mean,temperature_std,temperature_median,temperature_max,temperature_min,temperature_diff,temperature_tend,temperature_2018,temperature_2019,temperature_2020,temperature_2021,temperature_2022,temperature_2023,city,temperature_anomaly,temperature_slope,geometry,nomgeo
0,8b4995b81984fff,11,35.268092,6.180187,35.562500,43.671875,27.281250,16.390625,0.021635,36.120536,35.362500,35.800000,32.302083,34.197917,36.947917,CDMX,1.967899,-0.081565,"POLYGON ((2796916.249 831994.546, 2796934.314 ...",Azcapotzalco
1,8b4995b8db95fff,11,35.364662,6.510117,35.053571,44.648810,26.809524,17.839286,0.005952,36.392857,35.378571,35.328571,33.511905,34.601190,36.321429,CDMX,2.064469,-0.128741,"POLYGON ((2792221.707 835854.152, 2792239.775 ...",Azcapotzalco


In [8]:
data_mun_gdf = pd.merge(ndvi_gdf,temp_mun_gdf, on='hex_id')

# Show
print(data_mun_gdf.shape)
data_mun_gdf.head(2)

(326262, 38)


,hex_id,res_x,ndvi_mean,ndvi_std,ndvi_median,ndvi_max,ndvi_min,ndvi_diff,ndvi_tend,ndvi_2018,...,temperature_2019,temperature_2020,temperature_2021,temperature_2022,temperature_2023,city_y,temperature_anomaly,temperature_slope,geometry_y,nomgeo
0,8b4995bab64dfff,11,0.397782,0.121854,0.394442,0.501281,0.287628,0.213653,-0.004383,0.523185,...,31.238889,31.627778,29.476852,30.166667,32.132716,CDMX,-2.246099,-0.025557,"POLYGON ((2792526.900 834780.806, 2792544.968 ...",Azcapotzalco
1,8b4995b8db95fff,11,0.205466,0.054287,0.210134,0.266381,0.153338,0.113042,-0.001868,0.252257,...,35.378571,35.328571,33.511905,34.601190,36.321429,CDMX,2.064469,-0.128741,"POLYGON ((2792221.707 835854.152, 2792239.775 ...",Azcapotzalco


## Analyse by municipality

In [9]:
grouped_by = data_mun_gdf.groupby('nomgeo').agg({'temperature_anomaly':np.mean,
                                                 'temperature_slope':np.mean,
                                                 'ndvi_slope':np.mean})

# Show
grouped_by

,temperature_anomaly,temperature_slope,ndvi_slope
nomgeo,,,
Azcapotzalco,2.558288,-0.085331,-0.015913
Benito Juárez,0.538647,0.010821,-0.017280
Coyoacán,0.295450,0.025560,-0.023958
Cuajimalpa de Morelos,-5.196528,-0.224483,-0.036913
Cuauhtémoc,1.453197,-0.001921,-0.014286
Gustavo A. Madero,0.907614,-0.021615,-0.020411
Iztacalco,2.185320,-0.024725,-0.013513
Iztapalapa,1.839144,0.074023,-0.015421
La Magdalena Contreras,-2.359772,-0.155849,-0.028385
